In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import pycountry

# Analyze

In [ ]:
df = pd.read_parquet('all_lang_train.parquet')

In [ ]:
company_name = 'S.G. Exp.'
df[((df.name_1 == company_name) | (df.name_2 == company_name)) & (df.is_duplicate == 1)]

,name_1,name_2,is_duplicate


In [ ]:
df[((df.name_1 == company_name) | (df.name_2 == company_name)) & (df.is_duplicate == 0)]

In [ ]:
df['brace'] = df.name_1.apply(lambda x: True if '(' in x else False)

In [ ]:
df[df.brace == True]

,name_1,name_2,is_duplicate,brace
3014,Zhuli Zhiye (Shenzhen) Electronic Industrial C...,Nrc Industries Ltd.,0,True
3021,Weigao Plastic & Rubber Products(Shenzhen) Co....,"Shanghai Youbao Rubber Products Co., Ltd.",0,True
3042,Iida(Foshan) Industry?Co. Ltd.,"Fujian Qinglu Shoe Industry Co., Ltd.",0,True
3082,"China Merchants Heavy Industry (Shenzhen) Co.,...",Mona Industry Sarl,0,True
3091,Yusen Logistics (Y313),Tiger Logistics (India) Ltd. Mumbai,0,True
...,...,...,...,...
4385,"Wal Hee Trading (Shanghai) Co., Ltd.",Steps Trading,0,True
4386,Pcs (Pty) Ltd.,"Jay Precision Products (India) Pvt., Ltd.",0,True
4468,Ideal Carpet Industries (Pvt.) Ltd.,Cleartech Industries Inc.,0,True
4487,R1 International (Americas) Inc. 25,Bridgestone Americas Tire,0,True


Вывод = удаляем то что в скобках

In [ ]:
df['comma'] = df.name_1.apply(lambda x: True if ',' in x else False)

In [ ]:
df[df.comma == True][55:99]

Вывод = удаляем ','

# Preproc

In [ ]:
stopwords = ['Co.', 'Ltd.', 'Ltd', 'Private', 'International', 'Pacific', 'Pvt.',
             'Corp.',
             'Inc.', 'Sdn Bhd', 'Ltda',
             'S.A.C.', 'Inc.', 'A.S.',
             'Imp. & Exp.', 'Exp.', 'S.A.', 'S A.', 'S.A', 'Lp',
             '/Gmbh', 'Gmbh'
             'Sa De Cv', 'S.A. de C.V.'
             'Imp.', 'C.A.',
             'Llc', 'CO.', 'LTD.', 'PVT.', 'Asia']
stopwords = [st.replace('.', '\.') for st in stopwords]

In [ ]:
countries = [cnt.name for cnt in pycountry.countries if cnt.name not in ['Colombia']]

In [ ]:
def preproc(txt, stopwords):
    # remove stopwords
    # txt = txt.lower()
    for stopw in stopwords:
        txt = re.sub(stopw, ' ', txt)
    for cnt in countries:
        txt = re.sub(cnt, ' ', txt)

    txt = txt.replace(',', ' ')
    txt = txt.replace("'", ' ')
    txt = txt.replace(";", ' ')
    txt = txt.replace(":", ' ')
    txt = txt.replace('"', ' ')
    # remove geo info
    txt = re.sub('\(.+?\)', ' ', txt)
    # remove codes
    txt = re.sub('\w\d{1,}', ' ', txt)
    txt = txt.replace("(", ' ')
    txt = txt.replace(")", ' ')
    # replace consequent spaces
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt

In [ ]:
df = pd.read_parquet('languages.parquet')

In [ ]:
df

,Names,languages_langdetect
0,Iko Industries Ltd.,en
1,Apcotex Industries Ltd.,en
2,"Rishichem Distributors Pvt., Ltd.",en
3,Powermax Rubber Factory,en
4,Tress A/S,en
...,...,...
18017,Plastic Packaging (Pty) Ltd.,tl
18018,'Hengshui Mechanical & Electrical Building Co....,en
18019,"Jiangsu Baoli International Investment Co., Lt...",en
18020,Lanxess AG,ca


In [ ]:
df['name_preproc'] = df.Names.apply(lambda x: preproc(x, stopwords))

In [ ]:
df.to_parquet('df_names_preproc.parquet', compression='brotli')

In [ ]:
b = 790
df[44+b:60+b]

,Names,languages_langdetect,name_preproc
834,"Qingdao Wanyong Trade Co., Ltd.",tl,Qingdao Wanyong Trade
835,Dsa,tl,Dsa
836,Evonik Corp.,sl,Evonik
837,Uab Danspin,da,Uab Danspin
838,LG Chem Ltd.,de,LG Chem
839,"Qingdao Huateng International Logistics Co., Ltd.",en,Qingdao Huateng Logistics
840,Ibm,id,Ibm
841,Tailer S A,tr,Tailer S A
842,Trackon Promotional,en,Trackon Promotional
843,"Dongguan Yinaiba Rubber Co., Ltd.",id,Dongguan Yinaiba Rubber
